In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Creutzfeldt-Jakob_Disease/GSE87629'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide analysis of B and T cell gene expression during a six-week gluten challenge in patients with celiac disease"
!Series_summary	"Dietary gluten proteins (prolamins) from wheat, rye, and barley are the driving forces behind celiac disease, an organ-specific autoimmune disorder that targets both the small intestine and organs outside the gut. In the small intestine, gluten induces inflammation and a typical morphological change of villous atrophy and crypt hyperplasia. Gut lesions improve and heal when gluten is excluded from the diet and the disease relapses when patients consume gluten. Oral immune tolerance towards gluten may be kept for years or decades before breaking tolerance in genetically susceptible individuals. Celiac disease provides a unique opportunity to study autoimmunity and the transition in immune cells as gluten breaks oral tolerance. Seventy-three celiac disease patients on a long-term gluten-free diet ingested a known 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable


### Step 1: Check if gene expression data is likely available: Unknown Step

In [4]:
# Based on the series title and summary, it refers to DNA microarray analysis,
# which suggests that gene expression data is available.
is_gene_available = True


### Step 2: Variable Availability and Data Type Conversion: Unknown Step

In [5]:
# Step 2.1: Data Availability

# 'Creutzfeldt-Jakob_Disease' attribute - This information is not directly recorded or inferred in the given sample characteristics data.
# Thus, it is considered not available for this dataset.
trait_row = None 

# 'age' attribute - There is no information regarding age in the sample characteristics dictionary.
age_row = None

# 'gender' attribute - There is no information regarding gender in the sample characteristics dictionary.
gender_row = None

# Step 2.3: Data Type Conversion
def convert_trait(value):
    """Convert trait to binary; for our data this step is skipped as data is not available."""
    return None

def convert_age(value):
    """Convert age to continuous; for our data this step is skipped as data is not available."""
    return None

def convert_gender(value):
    """Convert gender to binary; for our data this step is skipped as data is not available."""
    return None

# Step 3: Save Metadata
save_cohort_info('GSE87629', './preprocessed/Creutzfeldt-Jakob_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction is skipped as trait_row is None
